In [1]:
import matplotlib.pyplot as plt
import os, shutil, copy
from ase import Atoms, Atom
from ase.units import Bohr, Hartree, kJ, mol
import pickle
import pandas as pd
from ase.io import read, write
import yaml

# Goal: Reproduction of Figure 2 in the [Published Paper](https://link.aps.org/doi/10.1103/PhysRevB.104.L161109)
but with the various epochs produced in the `sebshare` github folders. **These epochs are stored in the cluster, not on the github.**

3-row plot: WTMAD-2 (kcal/mol) for energy predictions, density errors $\times 10^3$, and the 'energy-density' error on bottom.

Per the paper, $$\mathrm{WTMAD-2} = \frac{1}{N} \sum_i^{55} N_i \frac{56.84 \mathrm{kcal/mol}}{\bar{|\Delta E|}_i}\mathrm{MAD}_i,$$ where $\mathrm{MAD}_i$ is the mean absolute energy deviations of a subset $i$ containing $N_i$ reactions, and $\bar{|\Delta E|}_i$ is the "inverse energy range" of a given subset.

**The parameters for different subsets of the GMTKN55 database come from [this paper](https://pubs.rsc.org/en/content/articlepdf/2017/cp/c7cp04913g) -- however, the [diet subset used in the published XCDiff paper](http://xlink.rsc.org/?DOI=C8CP05554H) has different weights due to the sampling.**


The density error metric is defined as $$\epsilon_{|n|}=\mathbb{E}\left[\frac{1}{N_e}\int_{\mathbf{r}} |n^{(i)}(\mathbf{r}) - n_\mathrm{ref}^{(i)}(\mathbf{r})| \right].$$

The combined energy-density error $$\mathcal{ED}_{|n|} = 2\left(\frac{1}{\mathrm{WTMAD-2}} + \frac{1}{f_E(\epsilon_{|n|})} \right)^{-1},$$ where $f_E(\epsilon_{|n|})=\gamma \epsilon_{|n|},$ with $\gamma = 1084.87$ kcal/mol (corresponding to the linear regression in Figure 3).

# DietGMTKN55-150 Dataset

The paper linked above lists [this github page](https://github.com/gambort/DietGMTKN55) as containing the relevant data.

In this repository, `GoodSamples/AllElements_150.yaml` contains all the relevant information for constructing the subset.

In [2]:
yp = '/home/awills/Documents/Research/DietGMTKN55/GoodSamples/AllElements_150.yaml'
with open(yp, 'r') as f:
    diet = yaml.safe_load(f)

The YAML entries are formatted as:
- GMTKN55 subset name
    - Index of molecule/reaction in that subset
        - Reference energy of the molecule/reaction (**in kcal/mol? unsure**)
        - The weight to use in MAD/WTMAD calculations
        - The species involved in the reference
            - The species index
                - Stoichiometry count (for when calculating the energy differences)
                - Molecule charge
                - Number of atoms in the molecule
                - A list of the atoms in the molecule
                - The array of positions corresponding to the atom array

In [3]:
diet['YBDE18']

{6: {'Energy': 51.74,
  'Weight': 1.15,
  'Species': {'me2s-ch2': {'Count': -1,
    'Charge': 0,
    'UHF': 0,
    'Number': 12,
    'Elements': ['C', 'H', 'H', 'S', 'C', 'C', 'H', 'H', 'H', 'H', 'H', 'H'],
    'Positions': [[-1.60787, -0.31747, 0.31592],
     [-2.31389, -0.88647, -0.26673],
     [-1.62499, -0.36553, 1.39673],
     [-0.16698, -0.06574, -0.457],
     [1.22081, -1.06918, 0.21515],
     [0.46703, 1.50357, 0.17771],
     [1.24595, -0.94516, 1.29755],
     [2.16545, -0.76153, -0.23353],
     [0.99677, -2.10238, -0.0378],
     [0.44313, 1.46775, 1.26621],
     [-0.20332, 2.27793, -0.18354],
     [1.48271, 1.66573, -0.17959]]},
   'me2s': {'Count': 1,
    'Charge': 0,
    'UHF': 0,
    'Number': 9,
    'Elements': ['S', 'C', 'C', 'H', 'H', 'H', 'H', 'H', 'H'],
    'Positions': [[0.0, 0.0, 0.66725],
     [0.0, 1.36148, -0.51768],
     [0.0, -1.36148, -0.51768],
     [-0.89163, 1.32617, -1.14153],
     [0.89163, 1.32617, -1.14153],
     [0.0, 2.28856, 0.05115],
     [-0.89163, 

In [4]:
#the subset of GMTKN55
diet150 = []
dkeys = sorted([i for i in diet.keys()])
for i, dk in enumerate(dkeys):
    subset = diet[dk]
    inds = sorted([i for i in subset.keys()])
    for j, iind in enumerate(inds):
        data = subset[iind]
        refen = data['Energy']
        refweight = data['Weight']
        specsdat = data['Species']
        specs = sorted([ii for ii in specsdat.keys()], key = lambda x: str(x))
        specsstr = [str(ii) for ii in specs]
        print("==================================")
        print("SUBSET {} -- INDEX {}".format(i, j))
        print("REFERENCE ENERGY: {} -- WEIGHT: {}".format(refen, refweight))
        print("----------------------------------")
        print("SPECIES INVOLVED IN SUBSET: {}".format(specs))
        print("..................................")
        for k, isp in enumerate(specs):
            specdat = specsdat[isp]
            specstr = specsstr[k]
            count = specdat['Count']
            spin = specdat['UHF'] #up - down electrons
            charge = specdat['Charge']
            nat = specdat['Number']
            els = specdat['Elements']
            pos = specdat['Positions']
            print("SPECIES: {} -- CHARGE: {} -- COUNT: {} -- NUMBER: {}".format(isp, charge, count, nat))
            print("CONSTITUENT ATOMS: {}".format(els))
            
            at = Atoms(els, positions=pos)
            at.info['spin'] = spin
            at.info['subset'] = dk
            at.info['subsetind'] = j
            at.info['species'] = isp
            at.info['count'] = count
            at.info['charge'] = charge
            at.info['refweight'] = refweight
            at.info['refen'] = refen
            at.info['energy'] = refen
            diet150 += [at]

SUBSET 0 -- INDEX 0
REFERENCE ENERGY: 2.632 -- WEIGHT: 30.99
----------------------------------
SPECIES INVOLVED IN SUBSET: ['H_g+x-t+', 'H_ttt']
..................................
SPECIES: H_g+x-t+ -- CHARGE: 0 -- COUNT: 1 -- NUMBER: 20
CONSTITUENT ATOMS: ['C', 'C', 'C', 'C', 'C', 'C', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H']
SPECIES: H_ttt -- CHARGE: 0 -- COUNT: -1 -- NUMBER: 20
CONSTITUENT ATOMS: ['C', 'C', 'C', 'C', 'C', 'C', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H']
SUBSET 0 -- INDEX 1
REFERENCE ENERGY: 3.083 -- WEIGHT: 30.99
----------------------------------
SPECIES INVOLVED IN SUBSET: ['H_ttt', 'H_x+g-g-']
..................................
SPECIES: H_ttt -- CHARGE: 0 -- COUNT: -1 -- NUMBER: 20
CONSTITUENT ATOMS: ['C', 'C', 'C', 'C', 'C', 'C', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H']
SPECIES: H_x+g-g- -- CHARGE: 0 -- COUNT: 1 -- NUMBER: 20
CONSTITUENT ATOMS: ['C', 'C', 'C', 'C', 'C', 'C', 'H',

In [6]:
for idx, at in enumerate(diet150):
    print(idx, at, at.info)

0 Atoms(symbols='C6H14', pbc=False) {'spin': 0, 'subset': 'ACONF', 'subsetind': 0, 'species': 'H_g+x-t+', 'count': 1, 'charge': 0, 'refweight': 30.99, 'refen': 2.632, 'energy': 2.632}
1 Atoms(symbols='C6H14', pbc=False) {'spin': 0, 'subset': 'ACONF', 'subsetind': 0, 'species': 'H_ttt', 'count': -1, 'charge': 0, 'refweight': 30.99, 'refen': 2.632, 'energy': 2.632}
2 Atoms(symbols='C6H14', pbc=False) {'spin': 0, 'subset': 'ACONF', 'subsetind': 1, 'species': 'H_ttt', 'count': -1, 'charge': 0, 'refweight': 30.99, 'refen': 3.083, 'energy': 3.083}
3 Atoms(symbols='C6H14', pbc=False) {'spin': 0, 'subset': 'ACONF', 'subsetind': 1, 'species': 'H_x+g-g-', 'count': 1, 'charge': 0, 'refweight': 30.99, 'refen': 3.083, 'energy': 3.083}
4 Atoms(symbols='ClFH', pbc=False) {'spin': 0, 'subset': 'AHB21', 'subsetind': 0, 'species': 6, 'count': 1, 'charge': -1, 'refweight': 2.53, 'refen': -25.52, 'energy': -25.52}
5 Atoms(symbols='Cl', pbc=False) {'spin': 0, 'subset': 'AHB21', 'subsetind': 0, 'species': '

In [5]:
len(diet150)

388

In [7]:
#write('/home/awills/Documents/Research/dpyscfl/data/dietgmtkn55-150/diet150.traj', diet150)

In [6]:
spins_dict = {
    'Al': 1,
    'B' : 1,
    'Li': 1,
    'Na': 1,
    'Si': 2 ,
    'Be':0,
    'C': 2,
    'Cl': 1,
    'F': 1,
    'H': 1,
    'N': 3,
    'O': 2,
    'P': 3,
    'S': 2
}


def get_spin(at):
    #if single atom and spin is not specified in at.info dictionary, use spins_dict
    print('======================')
    print("GET SPIN: Atoms Info")
    print(at)
    print(at.info)
    print('======================')
    if ( (len(at.positions) == 1) and not ('spin' in at.info) ):
        print("Single atom and no spin specified in at.info")
        spin = spins_dict[str(at.symbols)]
    else:
        print("Not a single atom, or spin in at.info")
        if type(at.info.get('spin', None)) == type(0):
            #integer specified in at.info['spin'], so use it
            print('Spin specified in atom info.')
            spin = at.info['spin']
        elif 'radical' in at.info.get('name', ''):
            print('Radical specified in atom.info["name"], assuming spin 1.')
            spin = 1
        elif at.info.get('openshell', None):
            print("Openshell specified in atom info, attempting spin 2.")
            spin = 2
        else:
            print("No specifications in atom info to help, assuming no spin.")
            spin = 0
    return spin

In [15]:
get_spin(diet150[0])

GET SPIN: Atoms Info
Atoms(symbols='C6H14', pbc=False)
{'spin': 0, 'subset': 'ACONF', 'subsetind': 0, 'species': 'H_g+x-t+', 'count': 1, 'charge': 0, 'refweight': 30.99, 'refen': 2.632, 'energy': 2.632}
Not a single atom, or spin in at.info
Spin specified in atom info.


0

In [10]:
diet150[0].info.get('spin')

0

In [7]:
atoms = diet150
atomic_set = []
for at in atoms:
    atomic_set += at.get_chemical_symbols()
atomic_set = list(set(atomic_set))
for s in atomic_set:
    assert s in list(spins_dict.keys()), "{}: Atom in dataset not present in spins dictionary.".format(s)


AssertionError: Ar: Atom in dataset not present in spins dictionary.

In [8]:
[s for s in atomic_set if s not in spins_dict.keys()]

['Ar', 'Br', 'Ne', 'Sb', 'Bi', 'Te', 'I']

In [ ]:
sd = {'Ar':0, #noble
     'Br':1, #one unpaired electron
     'Ne':0, #noble
     'Sb':3, #same column as N/P
     'Bi':3,
     'Te':'',
     'I':1 #one unpaired electron}